# Visualizing Snowflake Tables


This is a brief but complete example of how to visualize graphs represented by tables in Snowflake, using the Graph Visualization for Python library for Neo4j.

## Setup

We will start by installing the necessary Python library requirements.

In [ ]:
%pip install snowflake-snowpark-python # Requires Python version <= 3.11
%pip install neo4j-viz

We can now proceed to set up our connection to Snowflake by initializing a new session.
Please not that you may need more or fewer connection parameters depending on your Snowflake configuration.

In [ ]:
import os

from snowflake.snowpark import Session

# Configure according to your own setup
connection_parameters = {
    "account": os.environ.get("SNOWFLAKE_ACCOUNT"),
    "user": os.environ.get("SNOWFLAKE_USER"),
    "password": os.environ.get("SNOWFLAKE_PASSWORD"),
    "role": os.environ.get("SNOWFLAKE_ROLE"),
    "warehouse": os.environ.get("SNOWFLAKE_WAREHOUSE"),
}

session = Session.builder.configs(connection_parameters).create()

Now can we create a new Snowflake database where we can put our little example tables.
If you already have a database you want to use, you can skip this step.

In [ ]:
session.sql(
    "CREATE DATABASE IF NOT EXISTS nvl_example DATA_RETENTION_TIME_IN_DAYS = 1"
).collect()
session.sql("USE DATABASE nvl_example").collect()

## Creating tables

Next we will create a new table for the nodes in our graph, that will represent products of various categories.

In [ ]:
session.sql(
    "CREATE OR REPLACE TABLE products (id INT, name VARCHAR, category INT)"
).collect()

session.sql("""
INSERT INTO products VALUES
(1,  'Product 1',  1),
(2,  'Product 1A', 1),
(3,  'Product 1B', 1),
(4,  'Product 2',  2),
(5,  'Product 2A', 2),
(6,  'Product 2B', 2),
(7,  'Product 3',  3),
(8,  'Product 3A', 3),
(9,  'Product 3B', 3),
(10, 'Product 4',  4),
(11, 'Product 4A', 4),
(12, 'Product 4B', 4)
""").collect()

Some of the products, are "subproducts" of certain parent products.
We now create a table that encodes these "PARENT" relationships between the products.

In [ ]:
session.sql(
    "CREATE OR REPLACE TABLE parents (source INT, target INT, type VARCHAR)"
).collect()

session.sql("""
INSERT INTO parents VALUES
(2,  1,  'PARENT'),
(3,  1,  'PARENT'),
(5,  4,  'PARENT'),
(6,  4,  'PARENT'),
(8,  7,  'PARENT'),
(9,  7,  'PARENT'),
(11, 10, 'PARENT'),
(12, 10, 'PARENT')
""").collect()

## Fetching the data

Next we fetch our tables from Snowflake and convert them to pandas DataFrames.
Additionally, we rename the most of the table columns so that they are named according to the `neo4j-viz` API.

In [ ]:
products_df = (
    session.table("products")
    .to_pandas()
    .rename(columns={"ID": "id", "NAME": "caption"})
)
parents_df = (
    session.table("parents")
    .to_pandas()
    .rename(columns={"SOURCE": "source", "TARGET": "target", "TYPE": "caption"})
)

## Rendering the visualization
With only one command we can now create a `VisualizationGraph` from these tables representing nodes and relationships.
In order to enhance the visualization, we will also be utilizing the `color_nodes` function, which will assign a distinct color to each product category.

In [ ]:
from neo4j_viz.pandas import from_dfs

VG = from_dfs(products_df, parents_df)

# Using the default Neo4j color scheme
VG.color_nodes("CATEGORY")

Let us now render our graph, using only default render options.

In [ ]:
VG.render()

You can scroll to zoom in and out in the visualization, and click-and-drag nodes to move them.

## Cleanup

Lastly, we clean up the example database we created.

In [ ]:
session.sql("DROP DATABASE IF EXISTS nvl_example").collect()
session.close()